In [245]:
import pandas as pd
import numpy as np
import calendar

In [246]:
routes_per_month = pd.read_csv('../data/input/modified/routes_per_month.csv')
routes_per_month = routes_per_month[~routes_per_month['Month'].isin(['July', 'August'])] # dropping summer months

In [247]:
def count_weekdays(month, year):
    month_number = {
        'January': 1, 'February': 2, 'March': 3,
        'April': 4, 'May': 5, 'June': 6,
        'July': 7, 'August': 8, 'September': 9,
        'October': 10, 'November': 11, 'December': 12
    }[month]
    
    # Get the number of days in the month
    _, num_days = calendar.monthrange(year, month_number)
    
    # Initialize weekday count
    weekday_count = 0
    
    # Loop over each day of the month
    for day in range(1, num_days + 1):
        weekday = calendar.weekday(year, month_number, day)
        if weekday < 5:  # 0 to 4 are Monday to Friday
            weekday_count += 1
    
    return weekday_count

# Apply the function 
routes_per_month['Count of Weekdays'] = routes_per_month.apply(lambda row: count_weekdays(row['Month'], row['Year']), axis=1)
routes_per_month.head()

,Year,Month,Count of Routes,Count of Weekdays,Count of Days Off
0,2019,January,16417.0,23,NaN
1,2019,February,16402.0,20,NaN
2,2019,March,16422.0,21,NaN
3,2019,April,16429.0,22,NaN
4,2019,May,16430.0,23,NaN


In [248]:
data = pd.read_csv('../data/input/modified/DOE School Calendar Dates_edited.csv').drop(columns = ['Unnamed: 3', 'Unnamed: 4'])
data = data.replace('–', '-') 

start_dates, end_dates = [], []
for i in data['Dates']:
    if ('-' not in i):
        start_dates.append(pd.to_datetime(i))
        end_dates.append(pd.to_datetime(i))
    else:
        x = i.split(' ')
        start_dates.append(pd.to_datetime(' '.join([k.split('-')[0] if ('-' in k) else k for k in x])))
        end_dates.append(pd.to_datetime(' '.join([k.split('-')[1] if ('-' in k) else k for k in x])))
        
data['Start Date'] = start_dates
data['End Date'] = end_dates 

def get_year(school_year, month):
    if int(month) < 9:
        return int(school_year[:4]) + 1
    else:
        return int(school_year[:4]) 
    
data['End Date'] = pd.to_datetime(data['End Date'].dt.month.astype(str) + "/" + data['End Date'].dt.day.astype(str) + "/" + data.apply( (lambda x: get_year(x['School Year'], x['End Date'].month)), axis = 1).astype(str))

data['Start Date'] = pd.to_datetime(data['Start Date'].dt.month.astype(str) + "/" + data['Start Date'].dt.day.astype(str) + "/" + data.apply( (lambda x: get_year(x['School Year'], x['Start Date'].month)), axis = 1).astype(str))

covid = {'School Year': '2021-2022', 
        'Dates': '3/16/2020 - 9/13/2021', 
        'Reason': 'COVID', 
        'Start Date': pd.to_datetime('3/16/2020'), 
        'End Date': pd.to_datetime('9/13/2021')
        }

covid_df = pd.DataFrame([covid])
data = pd.concat([data, covid_df], ignore_index=True)

data['Closure'] = data['Reason'].apply( lambda x: 1 if  ( ("closed" in x) | 
                                    ("no students" in x) | 
                                    ("9-12 schools only" in x) | 
                                    ("remote" in x) | 
                                    ("hours early" in x) | 
                                    ("do not attend" in x) | 
                                    ("CLOSED" in x) | 
                                    ("Clerical Day" in x) | 
                                    ("staff development" in x) | 
                                    ("Students not in attendance" in x) | 
                                    ("Students will not" in x) | 
                                    ("will not be in attendance" in x) | 
                                    ("COVID" in x) | 
                                        ("Summer Recess" in x) ) else 0 )

In [249]:
school_closures_data = data[['Start Date', 'End Date', 'Closure']]

In [250]:
# Convert 'Start Date' and 'End Date' to datetime format
school_closures_data['Start Date'] = pd.to_datetime(school_closures_data['Start Date'])
school_closures_data['End Date'] = pd.to_datetime(school_closures_data['End Date'])

# Ensure 'Closure' days are counted correctly across multiple days if needed
dates = []
for index, row in school_closures_data.iterrows():
    if row['Closure'] == 1:
        date_range = pd.date_range(start=row['Start Date'], end=row['End Date'])
        dates.extend(date_range)

# Create a DataFrame with all closure dates
closure_dates = pd.DataFrame(dates, columns=['Date'])
closure_dates['Year'] = closure_dates['Date'].dt.year
closure_dates['Month'] = closure_dates['Date'].dt.strftime('%B')



/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_81770/1400628906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_closures_data['Start Date'] = pd.to_datetime(school_closures_data['Start Date'])
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_81770/1400628906.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_closures_data['End Date'] = pd.to_datetime(school_closures_data['End Date'])


In [251]:
# Add a column for isWeekday, where True indicates a weekday (Monday=0, ..., Sunday=6)
closure_dates['isWeekday'] = closure_dates['Date'].dt.weekday < 5

# Filter to keep only the weekdays
weekday_closures = closure_dates[closure_dates['isWeekday']]
weekday_closures = weekday_closures.drop_duplicates()

# Aggregate by month and year to count closure days, now only counting weekdays
weekday_closure_counts = weekday_closures.groupby(['Year', 'Month']).size().reset_index(name='Count of Days Off')

# Rough estimate of extra days "off" due to school starting and ending part of the way through these months (look for more exact way)
weekday_closure_counts.loc[weekday_closure_counts['Month'] == 'September', 'Count of Days Off'] += 6 # school tends to start on Sept 5-8
weekday_closure_counts.loc[weekday_closure_counts['Month'] == 'June', 'Count of Days Off'] += 3 # school tends to end around June 26-28



In [252]:
# Find number of days the bus operates each month
routes_per_month = pd.merge(routes_per_month[['Year','Month','Count of Routes','Count of Weekdays']], weekday_closure_counts, on=['Year', 'Month'], how='left')
routes_per_month.loc[routes_per_month['Count of Days Off'].isnull(), 'Count of Days Off'] = 0 # if no days off, switch to 0 so calculation is done correctly
routes_per_month['Count of Bus Operating Days'] = routes_per_month['Count of Weekdays'] - routes_per_month['Count of Days Off']
routes_per_month['Count of Bus Operating Days'] = routes_per_month['Count of Bus Operating Days'].clip(lower=0) # otherwise covid months would be negative

# Find the total number of trips each month
routes_per_month['Total Count of Trips'] = routes_per_month['Count of Bus Operating Days'] * routes_per_month['Count of Routes']
routes_per_month['Date'] = pd.to_datetime(routes_per_month['Year'].astype(str) + ' ' + routes_per_month['Month'] + ' 1')

In [253]:
# saving 

routes_per_month.to_csv('../data/output/total_bus_trips_per_month.csv')

In [254]:
routes_per_month

,Year,Month,Count of Routes,Count of Weekdays,Count of Days Off,Count of Bus Operating Days,Total Count of Trips,Date
0,2019,January,16417.0,23,3.0,20.0,328340.0,2019-01-01
1,2019,February,16402.0,20,6.0,14.0,229628.0,2019-02-01
2,2019,March,16422.0,21,0.0,21.0,344862.0,2019-03-01
3,2019,April,16429.0,22,6.0,16.0,262864.0,2019-04-01
4,2019,May,16430.0,23,1.0,22.0,361460.0,2019-05-01
5,2019,June,16437.0,20,6.0,14.0,230118.0,2019-06-01
6,2019,September,16657.0,21,8.0,13.0,216541.0,2019-09-01
7,2019,October,16852.0,23,2.0,21.0,353892.0,2019-10-01
8,2019,November,16909.0,21,4.0,17.0,287453.0,2019-11-01
9,2019,December,16951.0,22,7.0,15.0,254265.0,2019-12-01
